In [ ]:
!gdown --id 1UplpXVZtFZyzQ8SeeCPoetkJyNTlJD0w #Dataset 500

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1UplpXVZtFZyzQ8SeeCPoetkJyNTlJD0w
To: /content/DATASET.xlsx
100% 415k/415k [00:00<00:00, 79.0MB/s]


# **Library**

In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import library
import pandas as pd
import csv
import numpy as np
import nltk
import re

#from datasets import load_metric
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
# Random seed for consistency
np.random.seed(42)

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **Dataset**

In [ ]:
# download dataset
data = pd.read_excel('DATASET.xlsx')

In [ ]:
data.sample(5)

,web-scraper-order,web-scraper-start-url,judul,link-berita,narasi-berita,tanggal-berita,label
460,1666103656-178,https://www.kompas.com/tag/covid-19?sort=desc&...,"DJ Winky Wiryawan Positif Covid-19, Terpaksa J...",http://www.kompas.com/hype/read/2022/07/07/175...,Disjoki (DJ) sekaligus aktor Winky Wiryawan di...,"Kompas.com - 07/07/2022, 17:56 WIB",0
73,1665382040-632,https://covid19.go.id/id/p/hoax-buster?page=14...,[SALAH]: Mengonsumsi Kopi Pahit Dapat Mengatas...,https://covid19.go.id/id/artikel/2022/01/27/sa...,“*HOREE.. DAPAT ILMU BARU* ( *Atasi OMICRON* )...,27 Januari 2022,1
231,1666077352-10,https://covid19.go.id/id/p/hoax-buster?page=40...,[SALAH] “ke masjid tidak perlu memakai masker ...,https://covid19.go.id/id/artikel/2021/06/29/sa...,"“…… Pencegahan, pencegahannya jadi kitakan suk...",29 Juni 2021,1
175,1666065686-173,https://covid19.go.id/id/p/hoax-buster?page=33...,[SALAH] CNN Indonesia Menawarkan Hadiah agar W...,https://covid19.go.id/id/artikel/2021/07/25/sa...,“Tawaran by dunia media mainstream seperti ana...,25 Juli 2021,1
237,1666077342-6,https://covid19.go.id/id/p/hoax-buster?page=40...,[SALAH] Anies Baswedan Hanya Mengurusi Asbak d...,https://covid19.go.id/id/artikel/2021/06/27/sa...,“Orang lagi panik masalah Covid mengGila di Ja...,27 Juni 2021,1


In [ ]:
data.duplicated().sum()

0

In [ ]:
data.isna().sum()

web-scraper-order         0
web-scraper-start-url     0
judul                     0
link-berita              30
narasi-berita             0
tanggal-berita            6
label                     0
dtype: int64

In [ ]:
data.drop(['web-scraper-order','web-scraper-start-url', 'link-berita'], axis=1, inplace=True)
df = data[['judul', 'narasi-berita', 'label']]
df.sample(5)

,judul,narasi-berita,label
242,[SALAH] Malaysia Dapat Tambahan 10 Ribu Kuota ...,“Ngak mungkin!!! Sedangkan Malaysia mendapatka...,1
497,"Puluhan WNA di Bali Positif Covid-19, Dinkes U...",Puluhan warga negara asing (WNA) di Bali dilap...,0
139,[SALAH] Panggilan Telepon Informasi Seputar Va...,"“Baru saja, teman saya menerima telepon untuk ...",1
221,[SALAH] Ketersediaan Ventilator Cukup Memadai ...,“Seharusnya…3-4 bulan ke depan tidak perlu yan...,1
66,Awas Hoaks: Minum Kopi Pahit Terbukti Klinis S...,Beredar sebuah pesan berantai berisi narasi ya...,1


In [ ]:
df['label'].value_counts()

1    250
0    249
Name: label, dtype: int64

# **Text Preprocessing**

In [ ]:
stop_words = list(pd.read_csv(
    'https://raw.githubusercontent.com/ShinyQ/E-Wallet-Sentiment-Analysis_IFest-Unpad-2021/main/dataset/processed/stopwords_id_satya.txt', header=None)[0])

In [ ]:
# stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stopword
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [ ]:
df.drop(['judul'], axis=1, inplace=True)
df.sample(5)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,narasi-berita,label
368,Sejumlah siswa sekolah di Kota Palembang jalan...,0
420,Ahli Epidemiologi dari Griffith University Aus...,0
206,“…FDA (BPOM AMERIKA) telah menyetujui pengguna...,1
216,“Gubernur Sumut Instruksikan Seluruh masjid bu...,1
145,“Presiden umumkan bebas masker & kegiatan masy...,1


In [ ]:
def cleansing(text):
    text = str(text)
    print("Text awal :\n\n", text)

    # Mengubah setiap kata menjadi lowercase
    text = text.lower()

    # Menghapus Link Dengan Pattern http/https dan www
    text = re.sub(r'http\S+', '', text)
    text = re.sub('(@\w+|#\w+)', '', text)

    # Menghapus Tag HTML
    text = re.sub('<.*?>', '', text)

    # Menghapus Karakter Selain Huruf a-z dan A-Z
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Mengganti baris baru (enter) dengan spasi
    text = re.sub("\n", " ", text)

    # Menghapus Spasi Yang Lebih Dari Satu
    text = re.sub('(s{2,})', ' ', text)

    print("\nHasil Prerocessing :", text)

    text = stopword.remove(text)
    print("\nHasil Penghapusan Stopword :", text)

    text = stemmer.stem(text)
    print("\nHasil Stemming :", text)

    # Mengembalikan Hasil Preprocessing Text
    return text

In [ ]:
df['narasi-berita'] = df['narasi-berita'].apply(cleansing)

Text awal :

 Ada yang rela anaknya jadi Magneto junior?
.
Follow @teluuur
Follow @t3luuur”

Hasil Prerocessing : ada yang rela anaknya jadi magneto junior    follow  follow  

Hasil Penghapusan Stopword : yang rela anaknya jadi magneto junior    follow  follow  

Hasil Stemming : yang rela anak jadi magneto junior follow follow
Text awal :

 “Siap siap!!!

PPKM level 4 akan segera diberlakukan pada 24 desember!!!!!

Pentanyaanya????!!!!

Kok pemerintah sudah tau ya,bahwa kalau tgl 24 akan ada varian baru lagi

Hebat betul!!!!jenius banget otaknya!!!?

Apa bisnis PCR masih kurang…

Apa korupsinya masih kurang besar….

Curiga cuma akal akalan aja….supanya nanti menjelang puasa dan hariraya umat islam betul betul di lockdown lagi….supaya umat islam diam ditempat jaga jarak sholat,gak boleh jabat tangan dll….hebat!!!!!!!!!

Kita lihat aja predeksi ini kedepanya…….”

Hasil Prerocessing :  siap siap     ppkm level   akan segera diberlakukan pada    desember       pentanyaanya          kok p

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.head(30)

,narasi-berita,label
0,yang rela anak jadi magneto junior follow follow,1
1,siap siap ppkm level segera laku desember pent...,1
2,ingat mulai oktober pasien covid tanggung keme...,1
3,sinetron bajay bajuri thn sdh mau thn nyata ja...,1
4,bintang bollywood amir khan sempat hujat hanya...,1
5,moga allah mudah aamiin tak mampu bantu yg but...,1
6,moga manfaat ada salah laku konsumsi bahan teh...,1
7,corona ada china bukan sini sini cuma ada adak...,1
8,media covid perhati ph virus corona variasi hi...,1
9,sepupu teman positif corona duga kena jakarta ...,1


In [ ]:
df.to_excel('dataset_clean2.xlsx', index=False)